In [1]:
import pandas
from pandas import DataFrame
%matplotlib inline
import seaborn
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import cross_validation

import numpy
import math

In [2]:
train_all_df = DataFrame.from_csv("train.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
train_all_df.reset_index(inplace=True)
train_all_df.head()

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,Store,Date,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,2015-07-31,5,5263,555,1,1,0,1
1,2,2015-07-31,5,6064,625,1,1,0,1
2,3,2015-07-31,5,8314,821,1,1,0,1
3,4,2015-07-31,5,13995,1498,1,1,0,1
4,5,2015-07-31,5,4822,559,1,1,0,1


In [3]:
test_df = DataFrame.from_csv("test.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
test_df.reset_index(inplace=True)
test_df.head()

,Store,Date,Id,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday
0,1,2015-09-17,1,4,1,1,0,0
1,3,2015-09-17,2,4,1,1,0,0
2,7,2015-09-17,3,4,1,1,0,0
3,8,2015-09-17,4,4,1,1,0,0
4,9,2015-09-17,5,4,1,1,0,0


In [4]:
def featurize_basic(df):
    df = df.copy()
    def featurizeStateHoliday(x):
        if x == 'a':
            return 1
        elif x == 'b':
            return 2
        elif x == 'c':
            return 3
        else:
            return x
    df['StateHoliday'] = df['StateHoliday'].map(lambda x: featurizeStateHoliday(x))
    

    def featurizeDate(ts): 
        features = {}
        features['cal:dayOfMonth'] = ts.day
        features['cal:dayOfWeek'] = ts.dayofweek
        features['cal:dayofyear'] = ts.dayofyear
        features['cal:month'] = ts.month
        features['cal:year'] = ts.year
        features['cal:weekofyear'] =  ts.weekofyear
        features['cal:quarter'] = ts.quarter

        return features

    date_features_series = df['Date'].map(lambda x: featurizeDate(x))
    date_features = DataFrame.from_records(date_features_series.tolist(), index=df['Date'])

    del df['Date']
    
    df = pandas.concat([df, date_features.reset_index()], axis=1)

    
    return df

In [5]:

train_all_df = featurize_basic(train_all_df)

# Shuffle columnstrain_all_df.head()
columns = list(train_all_df.columns)
print (columns)
columns.remove("Sales")
columns = ["Sales"] + columns

train_all_df = train_all_df[columns]

train_all_df.head()

['Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'Date', 'cal:dayOfMonth', 'cal:dayOfWeek', 'cal:dayofyear', 'cal:month', 'cal:quarter', 'cal:weekofyear', 'cal:year']


,Sales,Store,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday,Date,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year
0,5263,1,5,555,1,1,0,1,2015-07-31,31,4,212,7,3,31,2015
1,6064,2,5,625,1,1,0,1,2015-07-31,31,4,212,7,3,31,2015
2,8314,3,5,821,1,1,0,1,2015-07-31,31,4,212,7,3,31,2015
3,13995,4,5,1498,1,1,0,1,2015-07-31,31,4,212,7,3,31,2015
4,4822,5,5,559,1,1,0,1,2015-07-31,31,4,212,7,3,31,2015


In [6]:
test_df = featurize_basic(test_df)
test_df.head()

,Store,Id,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,Date,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year
0,1,1,4,1,1,0,0,2015-09-17,17,3,260,9,3,38,2015
1,3,2,4,1,1,0,0,2015-09-17,17,3,260,9,3,38,2015
2,7,3,4,1,1,0,0,2015-09-17,17,3,260,9,3,38,2015
3,8,4,4,1,1,0,0,2015-09-17,17,3,260,9,3,38,2015
4,9,5,4,1,1,0,0,2015-09-17,17,3,260,9,3,38,2015


In [ ]:
del train_all_df['Sales']
del train_all_df['Customers']

In [12]:
train_all_df.to_csv("train_features_basic.csv", index=None)
test_df.to_csv("test_features_basic.csv", index=None)